In [1]:
# Deep Learning Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

# Data Manipulation and Analysis
import numpy as np
import pandas as pd
import collections # A module providing alternative data structures like named tuples, defaultdict, Counter, etc., compared to built-in Python containers.
import random

# Data Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn

# File and System Interaction
import glob
import os
from pathlib import Path
import shutil

# Scientific Computing and Math
import math
import cmath

# Date and Time Handling
import time
import datetime

# Linear Algebra
from torch import linalg as LA

# Neural Architecture
from torchinfo import summary


In [2]:
from python_scripts import generate_synthetic_data
from python_scripts import format_data
from python_scripts import dataset_processing
from python_scripts import forward_pass
from python_scripts import architecture

In [3]:
# Generate Data
M_train, M_Omega_train, M_test, M_Omega_test = generate_synthetic_data.generate(150, 300, 10, 40, 20, 0.45, 9)

print(f'M_train.shape: {M_train.shape}, M_Omega_train.shape: {M_Omega_train.shape}, M_test.shape: {M_test.shape}, M_Omega_test.shape: {M_Omega_test.shape}')

# Format and Save Data
format_data.format(M_train, M_Omega_train, M_test, M_Omega_test)

# Create DataLoaders
train_dataset = dataset_processing.ImageDataset(40, (150, 300), 0)
train_loader = data.DataLoader(train_dataset, batch_size = 5, shuffle = True)
test_dataset = dataset_processing.ImageDataset(5, (150, 300), 1)
test_loader = data.DataLoader(test_dataset, batch_size = 5, shuffle = True)

M_train.shape: (40, 150, 300), M_Omega_train.shape: (40, 150, 300), M_test.shape: (20, 150, 300), M_Omega_test.shape: (20, 150, 300)


In [4]:
for data in train_loader:
    print(data[0].shape, data[1].shape)

torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])
torch.Size([5, 150, 300]) torch.Size([5, 150, 300])


#### DataSet_Unfolded for Real World Sensing Data.py

In [5]:
# Setting up some global variables

ROOT = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/ConvHuberMC/HuberMC_Data'
TRY = '1st try'
SESSION = 'Session 1'

#### Model path of loading

Testing Training Loop

In [6]:
# Get parameters --> for convhubermc: c, lambda, sigma, mu, delta, tau
def get_default_param(gpu = True):
    params_net = {}
    params_net['layers'] = 6
    params_net['rank'] = 10

    params_net['initial_c'] = 1.345
    params_net['initial_lamda'] = 0.50
    params_net['initial_sigma'] = 0.67 # may not be learnable as then tau and lamda no longer needed

    params_net['initial_mu'] = 0.10
    params_net['initial_w'] = 0.2

    params_net['initial_delta'] = 0.40
    params_net['initial_tau'] = 0.21

    params_net['CalInGPU'] = gpu #whether to calculate in GPU
    params_net['iter'] = 5
    params_net['size1'] = 150
    params_net['size2'] = 300
    params_net['rank'] = 10
    return params_net

In [7]:
# Testing UnfoldedNetHuber
params_net = get_default_param()
model = architecture.UnfoldedNet_Huber(params = params_net)
model

UnfoldedNet_Huber(
  (huber_obj): Huber(
    (relu): ReLU()
    (sigmoid): Sigmoid()
  )
  (sig): Sigmoid()
  (relu): ReLU()
)

In [8]:
D, L = next(iter(train_loader))


In [9]:
# summary(model, input_size = [150, 300], col_names = ['input_size', 'output_size', 'num_params'])
model.forward(D[0])

V Multiprocess Done! 



AttributeError: 'generator' object has no attribute 'detach'

In [ ]:
# Some settings for visualisation
matplotlib.use('Agg')
%matplotlib inline

seed = 123
torch.manual_seed(seed)

# Set parameters (including hyperparameters) and setting for saving/logging data

hyper_param_net = training.get_hyperparameter_grid('HuberMC-Net', TrainInstances = 40, ValInstances = 20, BatchSize = 4, ValBatchSize = 4, num_epochs = 40, learning_rate = 0.0012)

params_net = get_default_param(True)

CalInGPU = params_net['CalInGPU']

q_list = [0.45]
db_list = [9.0]

for q in q_list:
  for db in db_list:
    ProjectName = TRY + ' ' + logs_and_results.get_current_time() + ' ' + hyper_param_net['Model'] + ' ' + 'Sampling Rate: ' + logs_and_results.get_q_str(q) + ' and DB ' + logs_and_results.get_noise_str(db)

    # Get log file
    logfile = logs_and_results.get_modularized_record(ProjectName, q, db, 'Logs', hyper_param_net, params_net, SESSION)
    log = open(logfile, 'w')
    print('Project Name: %s'%ProjectName)
    log.write('Project Name: %s\n'%ProjectName)

    # Get Model
    net = training.get_model(params_net, hyper_param_net, log)
    print('Parameters = \n%s\n'%str(params_net))
    log.write('params_net=\n%s\n\n'%str(params_net))

    #Loading data and creating dataloader for both test and training
    print('Loading Data phase...')
    print('----------------')
    log.write('Loading phase...\n')
    log.write('----------------\n')
    shape_dset = (params_net['size1'], params_net['size2'])
    
    train_dataset = dataset_processing.ImageDataset(round(hyper_param_net['TrainInstances']), shape_dset, 0, q, db)
    train_loader = data.DataLoader(train_dataset,batch_size = hyper_param_net['BatchSize'], shuffle = True)
    val_dataset = dataset_processing.ImageDataset(round(hyper_param_net['ValInstances']), shape_dset, 1, q, db)
    test_loader = data.DataLoader(val_dataset, batch_size = hyper_param_net['ValBatchSize'], shuffle = True)

    print('Finished loading.\n')
    log.write('Finished loading.\n\n');

    # Some additional settings for training including loss, optimizer,
    floss = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr = hyper_param_net['Lr'])
    scheduler2 =  torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma = 0.97, verbose = True)

    # Array for recording parameter values after each layer for each epoch etc
    outputs_L = hubermc.to_var(torch.zeros([shape_dset[0], shape_dset[1]]), CalInGPU) 
    lossmean_vec = np.zeros((hyper_param_net['Epochs'], ))
    lossmean_val_vec = np.zeros((hyper_param_net['Epochs'], ))

    lamda, mu, c, sigma, tau = net.getexp_LS()

    lamda_vec = np.zeros((hyper_param_net['Epochs'], net.layers))
    mu_vec = np.zeros((hyper_param_net['Epochs'], net.layers))
    c_vec = np.zeros((hyper_param_net['Epochs'], net.layers))
    db_vec = np.zeros((hyper_param_net['Epochs'], net.layers))
    tau_vec = np.zeros((hyper_param_net['Epochs'], net.layers))

    # dummy variable to monitor and record progress for loss
    minloss = np.inf

    for epoch in range(hyper_param_net['Epochs']):
      print(f'Epoch: {epoch + 1}, {logs_and_results.get_current_time()}, \n')
      log.write('\n' + logs_and_results.get_current_time() + '\n')

      # Train and Test Steps. (Record every 5 epochs)
      if (epoch + 1) % 5 == 0:
          print('Loading and calculating training batches...')
          log.write('Loading and calculating training batches...\n')
          startime = time.time()
          loss_mean, loss_lowrank_mean = training.train_step(net, train_loader, floss, optimizer, CalInGPU, hyper_param_net['TrainInstances'], hyper_param_net['BatchSize']) # remove alpha from train func
          endtime = time.time()
          print('Training time is %f'%(endtime - startime))
          log.write('Training time is %f\n'%(endtime - startime))

          print('Loading and calculating validation batches...')
          log.write('Loading and calculating validation batches...\n')
          startime = time.time()
          loss_val_mean, loss_val_lowrank_mean = training.test_step(net, test_loader, floss, CalInGPU, hyper_param_net['ValInstances'], hyper_param_net['ValBatchSize'])
          endtime = time.time()
          print('Test time is %f'%(endtime - startime))
          log.write('Test time is %f\n'%(endtime - startime))

      else:
        loss_mean, loss_lowrank_mean = training.train_step(net, train_loader, floss, optimizer, CalInGPU, hyper_param_net['TrainInstances'], hyper_param_net['BatchSize'])
        loss_val_mean, loss_val_lowrank_mean = training.test_step(net, test_loader, floss, CalInGPU, hyper_param_net['ValInstances'], hyper_param_net['ValBatchSize'])

      # Update Record and Parameters
      lossmean_vec[epoch] = loss_mean
      lossmean_val_vec[epoch] = loss_val_mean

      lamda, mu, c, sigma, tau = net.getexp_LS()

      lamda_vec[epoch, :] = lamda
      mu_vec[epoch, :] = mu
      c_vec[epoch, :] = c
      db_vec[epoch, :] = db
      tau_vec[epoch, :] = tau

      # Update Log after every 5 epochs. Make a plot of MSE against epochs every 5 epochs. Save Model in whole/dict form every five epochs.
      if (epoch + 1) % 5 == 0:
        print(f"Saving Whole Model at Epochs: [{epoch + 1}/{hyper_param_net['Epochs']}]")
        model_whole_path = logs_and_results.get_modularized_record(ProjectName, q, db, 'Saved Models - Whole', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        torch.save(net, model_whole_path)
        print(f"Saving Model Dict at Epochs: [{epoch + 1}/{hyper_param_net['Epochs']}]")
        model_state_dict_path = logs_and_results.get_modularized_record(ProjectName, q, db, 'Saved Models - Dict', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        torch.save(net.state_dict(), model_state_dict_path)

        print('Epoch [%d/%d], Lossmean:%.5e, Validation lossmean:%.5e'
              %(epoch + 1, hyper_param_net['Epochs'], loss_mean, loss_val_mean))
        print('loss_lowrank_mean', loss_lowrank_mean)
        print('loss_val_lowrank_mean', loss_val_lowrank_mean)

        log.write('loss_lowrank_mean %.5e\n' %(loss_lowrank_mean))
        log.write('loss_val_lowrank_mean %.5e\n' %(loss_val_lowrank_mean))
        log.write('Epoch [%d/%d], Lossmean:%.5e, Validation lossmean:%.5e\n'
              %(epoch + 1, hyper_param_net['Epochs'], loss_mean, loss_val_mean))
        np.set_printoptions(precision = 3)

        log.write('lamda: '+ str(lamda)+'\n')
        log.write('mu: '+ str(mu)+'\n')
        log.write('c: '+ str(c)+'\n')
        log.write('sigma: '+ str(sigma)+'\n')
        log.write('tau: '+ str(tau)+'\n')

        if True or loss_val_mean<minloss:
          print('saved at [epoch%d/%d]'%(epoch + 1, hyper_param_net['Epochs']))
          log.write('saved at [epoch%d/%d]\n' %(epoch + 1, hyper_param_net['Epochs']))
          minloss = min(loss_val_mean, minloss)

        # Plotting MSE vs Epoch and Saving it

        # Get Directory where we have to save the plot
        dir = logs_and_results.get_modularized_record(ProjectName, q, db, 'Plots', hyper_param_net, params_net, SESSION, current_epoch = epoch + 1)
        epochs_vec = np.arange(0, hyper_param_net['Epochs'], 1)
        logs_and_results.plot_and_save_mse_vs_epoch(epochs_vec, lossmean_vec, hyper_param_net, lossmean_val_vec, dir, epoch)

    # Finish off by observing the minimum loss on validation set

    #Print min loss
    print('\nmin Loss = %.4e'%np.min(lossmean_val_vec))
    log.write('\nmin Loss = %.4e\n'%np.min(lossmean_val_vec))
    log.close()

Project Name: 5st try 2023-10-04 00:11:23 ConvMC-Net Sampling Rate: 50.0% and Noise Variance 3.0
Configuring Network...

Instantiating Model...

Model Instantiated...

Parameters = 
{'layers': 5, 'kernel': [(3, 1), (3, 1), (3, 1), (3, 1), (3, 1)], 'initial_mu_inverse': 0.0, 'initial_y1': 0.8, 'coef_mu_inverse': 0.36, 'CalInGPU': True, 'size1': 400, 'size2': 500}

Loading Data phase...
----------------
Finished loading.

Adjusting learning rate of group 0 to 1.2000e-02.
Epoch: 1, 2023-10-04 00:11:23, 



RuntimeError: ignored